# Collab stuff to link git and google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/
git_token =   #Generate an access token
username = "rizwan-nato"
repository = "TP_NLP"
!git config --global user.email "rizwanato@hotmail.fr"
!git config --global user.name "Rizwan Nato"
#!git clone https://{git_token}@github.com/{username}/{repository}
%cd /content/drive/MyDrive/Github/TP_NLP/TP1
#!git pull

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Github
/content/drive/MyDrive/Github/TP_NLP/TP1


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from skipGram import *

In [4]:
from scipy.stats import pearsonr
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

In [5]:
def run_training(sg, Number_of_epochs, Learning_rate_Schedule, early_stopping = 1e-3):
  for i, lr in zip(range(Number_of_epochs), Learning_rate_Schedule):
    sg.train(epochs=1, lr=lr)
    pairs = loadPairs("simlex.csv")
    Y_true = []
    Y_pred = []
    Y_true2 = []
    Y_pred2 = []
    for a, b, y_true in pairs:
      pred, unknown = sg.similarity(a,b)
      Y_pred.append(pred)
      Y_true.append(y_true)
      if not unknown:
        Y_pred2.append(pred)
        Y_true2.append(y_true)
    Y_pred = np.array(Y_pred)
    Y_true = np.array(Y_true)
    Y_pred2 = np.array(Y_pred2)
    Y_true2 = np.array(Y_true2)

    pairs = loadPairs("simlex.csv")
    Y_true_combine = []
    Y_pred_combine = []
    Y_true2_combine = []
    Y_pred2_combine = []
    for a, b, y_true in pairs:
      pred, unknown = sg.similarity(a,b, combine_embed=True)
      Y_pred_combine.append(pred)
      Y_true_combine.append(y_true)
      if not unknown:
        Y_pred2_combine.append(pred)
        Y_true2_combine.append(y_true)
    Y_pred_combine = np.array(Y_pred_combine)
    Y_true_combine = np.array(Y_true_combine)
    Y_pred2_combine = np.array(Y_pred2_combine)
    Y_true2_combine = np.array(Y_true2_combine)
    print(f"There are {len(Y_true) - len(Y_true2)} unknown words")
    print("     > Correlation on all words:", pearsonr(Y_true, Y_pred), pearsonr(Y_true_combine, Y_pred_combine))
    print("     > Correlation on known words:", pearsonr(Y_true2, Y_pred2), pearsonr(Y_true2_combine, Y_pred2_combine))
    if i > 0:
      if sg.loss[-2] - sg.loss[-1] < early_stopping:
        print("Early stopping..")
        break;
  return pearsonr(Y_true, Y_pred), pearsonr(Y_true_combine, Y_pred_combine)

In [8]:
Learning_rate_Schedule =  [1e-2]*100
Number_of_epochs = 100

In [9]:
correlation_dict = {}
number_of_line = 5000
sentences_no_lemma = text2sentences("train_full_24.txt", number_of_line=number_of_line, lemma=False)
sentences_with_lemma = text2sentences("train_full_24.txt", number_of_line=number_of_line, lemma=True)

100%|██████████| 5000/5000 [01:25<00:00, 58.28it/s]


In [ ]:
# with open("correlation5000.dict", 'wb') as f:
#   pickle.dump(correlation_dict, f)

In [10]:
with open("correlation5000.dict", "rb") as f:
  correlation_dict = pickle.load(f)

In [11]:
correlation_dict

{}

In [ ]:
nEmbed = 100
for winSize in [9,5,3]:
  for minCount in [5,2]:
    for i, sentence in enumerate([sentences_no_lemma, sentences_with_lemma]):
      print("############################################################################")
      print(f"Training model: miniCount {minCount} nEmbed {nEmbed} winSize {winSize} Lemma {i==1}")
      if f"miniCount {minCount} nEmbed {nEmbed} winSize {winSize} Lemma {i==1} Not combined" in correlation_dict:
        print("Already Trained Skipping... \n")
        continue
      sg = SkipGram(sentences=sentence, minCount=minCount, nEmbed=nEmbed, winSize=winSize, negativeRate=5)
      correlation, correlation_combine = run_training(sg, Number_of_epochs, Learning_rate_Schedule)
      correlation_dict[f"miniCount {minCount} nEmbed {nEmbed} winSize {winSize} Lemma {i==1} Not combined"] =  correlation
      correlation_dict[f"miniCount {minCount} nEmbed {nEmbed} winSize {winSize} Lemma {i==1} Combined"] =  correlation_combine
      print("     Saving Correlation")
      with open("correlation5000.dict", 'wb') as f:
          pickle.dump(correlation_dict, f)
      word1 = "president"
      Similarity = []
      for word2 in sg.vocab:
        Similarity.append((word2, sg.similarity(word1,word2)))
      b = sorted(Similarity, key=lambda Similarity:Similarity[1])
      print("\n")
      print("Words closest and furthest away from president")
      print(b[-20:])
      print(b[:20])
      print("\n")
      print("\n")
    # sg.save(f"Models/NoLemma/miniCount{minCount}_nEmbed{nEmbed}_winSize{winSize}.model")

############################################################################
Training model: miniCount 5 nEmbed 100 winSize 9 Lemma False
Training Epoch 1


  9%|▉         | 447/5000 [00:11<02:23, 31.69it/s]

In [ ]:
word1 = "president"
Similarity = []
for word2 in sg.vocab:
  Similarity.append((word2, sg.similarity(word1,word2)))

b = sorted(Similarity, key=lambda Similarity:Similarity[1])
print(b[-50:])
print(b[:50])